In [60]:
# Import relevant libraries
import vlc
from pathlib import Path, WindowsPath
import pandas as pd

### Test playing a video

In [6]:
# Define parent directory for shows
TV_PATH = Path("./data/TV Shows/")

In [17]:
# Get episode 1 of show
test_vid = next(next(TV_PATH.glob("*")).glob("*"))


In [18]:
# Test playing program with VLC
media = vlc.MediaPlayer(test_vid)
media.play()

0

In [19]:
# Stop when cell is run
media.stop()

### Generate Playlist File

In [93]:
def generate_playlist_scheme(file_name : str):
    # Grab folder name for each video
    show_folders = [folder for folder in TV_PATH.glob("*")]

    # Create and populate dataframe
    show_folder_df = pd.DataFrame(columns=["show_path", "frequency"])
    show_folder_df["show_path"] = show_folders

    # Make relative to TV_PATH and remove playlist row
    show_folder_df["show_path"] = show_folder_df["show_path"].apply(lambda x: x.relative_to(TV_PATH))
    show_folder_df = show_folder_df[show_folder_df["show_path"] != WindowsPath('.playlists')]

    # Create playlist file
    show_folder_df.to_csv(TV_PATH.as_posix() + "/.playlists/" + file_name + ".csv")

In [94]:
generate_playlist_scheme("playlist")

### Test Reading Playlist File and Creating Random Subset

In [107]:
def load_playlist_scheme(file_name : str) -> pd.DataFrame:
    # Grab playlist scheme from file
    playlist = pd.read_csv(TV_PATH.as_posix() + "/.playlists/" + file_name + ".csv", index_col=0)
    return playlist

In [108]:
load_playlist_scheme("playlist")

,show_path,frequency
1,4543,NaN
2,Adventure Time (2010) Season 1-10 S01-S10 + Ex...,NaN
3,Hercules - The Legendary Journeys (1994)(tvdb-72),NaN
4,Star Wards - The Clone Wars (2008),NaN


In [ ]:
# Set constraints
MAX_LENGTH_MINS = 120
